# Импорты

In [1]:
!pip install catboost -q
!pip install ipynbname -q
!pip install optuna

^C


In [3]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, make_scorer, f1_score
from sklearn.utils.class_weight import compute_class_weight

import optuna

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files
from google.colab import drive
import ipynbname
import shutil

import datetime
import pytz

import os

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Константы

In [4]:
SEED = 42
MODE = 'validation'

# paths
DRIVE_PATH = './gdrive/MyDrive/ml_hits_1_module'
TRAIN_FILE_NAME = 'train.csv'
TEST_FILE_NAME = 'test.csv'
SUBMISSION_FILE_NAME = 'sample_submission.csv'
DATA_PATH = DRIVE_PATH + '/data'
SUBS_PATH = DRIVE_PATH + '/submits'
NOTEBOOKS_PATH = DRIVE_PATH + '/notebooks'

# Вспомогательные функции

In [5]:
def set_all_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

set_all_seeds(SEED)

# Загрузка данных

In [6]:
path = os.path.join(DATA_PATH, TRAIN_FILE_NAME)
train = pd.read_csv(path)
print(train.shape)
train.sample(5)

(8693, 14)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
304,0337_02,Mars,False,F/63/S,TRAPPIST-1e,19.0,False,417.0,349.0,634.0,3.0,1057.0,Weros Perle,True
2697,2891_01,Earth,False,G/460/S,TRAPPIST-1e,18.0,False,4.0,904.0,0.0,0.0,1.0,Gleney Ortinericey,False
8424,8998_01,Earth,True,G/1449/S,TRAPPIST-1e,41.0,False,0.0,0.0,0.0,0.0,0.0,Gerry Englence,False
1672,1771_01,Earth,False,G/291/P,TRAPPIST-1e,35.0,False,0.0,338.0,436.0,NaN,0.0,Antone Cardner,True
8458,9034_02,Europa,True,D/288/P,TRAPPIST-1e,43.0,False,0.0,0.0,0.0,0.0,0.0,Errairk Crakete,True


In [7]:
path = os.path.join(DATA_PATH, TEST_FILE_NAME)
test = pd.read_csv(path)
print(test.shape)
test.sample(5)

(4277, 13)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
2824,6244_01,Europa,False,A/61/P,TRAPPIST-1e,37.0,False,213.0,5.0,0.0,439.0,1190.0,Atino Spopting
2117,4583_03,Mars,False,D/146/S,TRAPPIST-1e,50.0,False,953.0,1628.0,1184.0,0.0,1.0,Per Parte
1332,2850_03,Mars,False,E/192/S,TRAPPIST-1e,26.0,False,1208.0,90.0,1.0,0.0,0.0,Shog Nuthe
3749,8183_01,Earth,False,E/526/P,TRAPPIST-1e,51.0,False,0.0,0.0,7.0,365.0,625.0,Jayne Hayder
2827,6255_01,Europa,True,B/212/P,TRAPPIST-1e,42.0,False,0.0,0.0,0.0,0.0,0.0,Schird Siccertful


# Обработка данных

In [8]:
train.head(50)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


# EDA

## Преобразование фичей

In [9]:
!pip install gender-guesser
import gender_guesser.detector as gender # Предсказание пола

gender_detector = gender.Detector()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.3/379.3 kB 3.5 MB/s eta 0:00:00


In [10]:
def predict_gender(name):
    # Получаем первую часть имени (на случай, если имя состоит из нескольких слов)
    first_name = name
    # Определяем пол
    gender_pred = gender_detector.get_gender(first_name)
    # Упрощаем результат: 'male' или 'female', остальное считаем 'unknown'
    if gender_pred in ['male', 'mostly_male']:
        return 'male'
    elif gender_pred in ['female', 'mostly_female']:
        return 'female'
    else:
        return 'unknown'

In [11]:
def process_features(df):
    # Разбиваем на PassengerId на PassengerGroup и PassengerNumberInGroup
    df['PassengerGroup'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
    df['PassengerNumberInGroup'] = df['PassengerId'].apply(lambda x: int(x.split('_')[1]))

    # Разбиваем Cabin на Deck, Num, Side
    df['Cabin'] = df['Cabin'].astype(str)
    df['Deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if x != 'nan' else np.nan)
    df['Num'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if x != 'nan' else np.nan)
    df['Side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if x != 'nan' else np.nan)

    # Разбиваем Name на Surname и Name
    df['Name'] = df['Name'].fillna('Unknown Unknown') # Если имя неизвестно
    df['Surname'] = df['Name'].apply(lambda x: x.split()[1])
    df['Name'] = df['Name'].apply(lambda x: x.split()[0])

    # Придумываем новые фичи

    # Сколько всего заплатил пассажир
    df['PaidSum'] = (df['RoomService'] + df['FoodCourt'] +
                     df['ShoppingMall'] + df['Spa'] +
                     df['VRDeck'])
    # Платил ли пассажир за доп услуги
    df['PaidExtra'] = (df['RoomService'] + df['FoodCourt'] +
                       df['ShoppingMall'] + df['Spa'] +
                       df['VRDeck'] != 0.0)
    # Сколько человек в группе пассажира
    df['PassengersCountInGroup'] = df.groupby('PassengerGroup')['PassengerNumberInGroup'].transform('max')
    # В одиночку ли путешествует
    df['IsAlone'] = (df['PassengersCountInGroup'] == 1) & (df['PassengerNumberInGroup'] == 1)
    # Путешествует ли с родственниками
    df['IsTravelWithFamily'] = df.groupby('PassengerGroup')['Surname'].transform(lambda x: (x == x.iloc[0]).sum() > 1)
    # Пытаемся понять пол по имени
    df['Gender'] = df['Name'].apply(predict_gender)

    # Удаляем лишнее
    cols2drop = ['PassengerId', 'Cabin', 'Name', 'Surname']
    df = df.drop(cols2drop, axis=1, errors='ignore')

    return df

In [12]:
train = process_features(train)

### Применение изменений

In [13]:
target = ['Transported']
cat_features = ['HomePlanet', 'CryoSleep', 'Destination',
                'VIP', 'Deck', 'Side', 'PaidExtra', 'IsAlone',
                'IsTravelWithFamily', 'Gender']

filtered_features = [i for i in train.columns if (i not in target)]
num_features = [i for i in filtered_features if i not in cat_features]


print('cat_features :', len(cat_features), cat_features)
print('num_features :', len(num_features), num_features)
print('targets', target)

cat_features : 10 ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side', 'PaidExtra', 'IsAlone', 'IsTravelWithFamily', 'Gender']
num_features : 11 ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'PassengerGroup', 'PassengerNumberInGroup', 'Num', 'PaidSum', 'PassengersCountInGroup']
targets ['Transported']


In [14]:
train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,PassengerNumberInGroup,Deck,Num,Side,PaidSum,PaidExtra,PassengersCountInGroup,IsAlone,IsTravelWithFamily,Gender
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,...,1,B,0.0,P,0.0,False,1,True,False,unknown
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,...,1,F,0.0,S,736.0,True,1,True,False,unknown
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,...,1,A,0.0,S,10383.0,True,2,False,True,unknown
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,...,2,A,0.0,S,5176.0,True,2,False,True,unknown
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,...,1,F,1.0,S,1091.0,True,1,True,False,male


## Пропуски

In [15]:
train.isna().sum().sort_values(ascending=False)

,0
PaidSum,908
CryoSleep,217
ShoppingMall,208
VIP,203
HomePlanet,201
Deck,199
Num,199
Side,199
VRDeck,188
FoodCourt,183


In [16]:
train.shape

(8693, 22)

Пробуем заменить

In [17]:
def replace_nans(df):
  # Численные фичи заполняем медианами
  for feature in num_features:
      train[feature] = train[feature].astype(float)

  median_values = train[num_features].median()
  df[num_features] = df[num_features].fillna(median_values)

  # Несбалансированные категориальные фичи заполняем самым популярным классом
  disbalanced_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Gender']

  for feature in disbalanced_features:
      df[feature] = df[feature].fillna(train[feature].mode()[0])

  # Остальные категориальные фичи заполняем рандомным классом
  balanced_features = [i for i in cat_features if i not in disbalanced_features]

  for feature in balanced_features:
      non_nan_values = df[feature].dropna().unique()

      # Заменяем NaN на случайное значение из non_nan_values
      df[feature] = df[feature].apply(lambda x: np.random.choice(non_nan_values) if pd.isna(x) else x)

  return df

In [18]:
train = replace_nans(train)

<ipython-input-17-e085b354fe4b>:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[feature] = df[feature].fillna(train[feature].mode()[0])


## Графики

### Офигеваем с выбросов

In [ ]:
train.hist(figsize=(20,15))

Выбросы?

Пробуем удалить

In [ ]:
# def delete_outliers(df, quantile_size=0.01):
#     for feature in num_features:
#         Q1 = df[feature].quantile(quantile_size)
#         Q3 = df[feature].quantile(1 - quantile_size)
#         IQR = Q3 - Q1

#         lower_bound = Q1 - 1.5 * IQR
#         upper_bound = Q3 + 1.5 * IQR

#         df = df[(df[feature] >= lower_bound) & (df[feature] <= upper_bound)]

#     return df

In [ ]:
# train.shape[0]

8693

In [ ]:
# train = delete_outliers(train)
# train.shape[0]

8648

### Смотрим категориальные фичи

In [ ]:
n_cols = 3
n_rows = (len(cat_features) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5))
axes = axes.flatten()

for i, feature in enumerate(cat_features):
    sns.countplot(x=feature, data=train, ax=axes[i])
    axes[i].set_title(f'Распределение фичи {feature}')
    axes[i].tick_params(axis='x', rotation=45)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

### Смотрим че там с таргетом

In [ ]:
train['Transported'].value_counts()

Таргет сбалансирован ура

In [ ]:
from sklearn.preprocessing import LabelEncoder
def show_corr_matrix(data):
    data_new = data.copy()
    for col in cat_features:
        data_new[col] = LabelEncoder().fit_transform(data[col])

    sns.heatmap(data_new.corr())

In [ ]:
show_corr_matrix(train)

In [ ]:
pd.crosstab(
    train['CryoSleep'],
    train['Transported']
).style.background_gradient('RdYlGn')

In [ ]:
sns.relplot(
    data=train,
    kind='line',
    x='RoomService',
    y='Spa',
    hue='Transported',
    aspect=4
).set_xticklabels(rotation=45, horizontalalignment='right', step=2)

# Обучение

In [19]:
X = train.drop(target, axis=1, errors='ignore')
y = train[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=SEED)

In [21]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'verbose': False,
        'cat_features': cat_features,
        'random_state': SEED
    }

    model = CatBoostClassifier(**params)

    model.fit(
        X_train,
        y_train,
        eval_set=(X_test, y_test),
        verbose=150,
        plot=False
    )

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [22]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-03-06 21:12:33,954] A new study created in memory with name: no-name-5842ea0d-7d6f-4891-9955-06702e2188a2


0:	learn: 0.6857008	test: 0.6866738	best: 0.6866738 (0)	total: 145ms	remaining: 1m 56s
150:	learn: 0.3886267	test: 0.4294319	best: 0.4294319 (150)	total: 10.2s	remaining: 44.2s
300:	learn: 0.3326350	test: 0.3906823	best: 0.3906739 (298)	total: 19.9s	remaining: 33.3s
450:	learn: 0.2988381	test: 0.3737993	best: 0.3730106 (433)	total: 28.3s	remaining: 22.2s
600:	learn: 0.2740109	test: 0.3676625	best: 0.3671386 (569)	total: 31.9s	remaining: 10.8s
750:	learn: 0.2526570	test: 0.3670927	best: 0.3641965 (691)	total: 37.5s	remaining: 2.69s


[I 2025-03-06 21:13:13,342] Trial 0 finished with value: 0.8160919540229885 and parameters: {'n_estimators': 805, 'learning_rate': 0.011351308425298882, 'depth': 8, 'l2_leaf_reg': 0.004760812477255313, 'random_strength': 0.035521031002808304, 'bagging_temperature': 0.6652113459798238, 'border_count': 147}. Best is trial 0 with value: 0.8160919540229885.


804:	learn: 0.2459414	test: 0.3685289	best: 0.3641965 (691)	total: 38.7s	remaining: 0us

bestTest = 0.3641964577
bestIteration = 691

Shrink model to first 692 iterations.
0:	learn: 0.6827088	test: 0.6846539	best: 0.6846539 (0)	total: 14.1ms	remaining: 10.1s
150:	learn: 0.4006123	test: 0.4254207	best: 0.4254207 (150)	total: 1.85s	remaining: 6.91s
300:	learn: 0.3640538	test: 0.3938601	best: 0.3938601 (300)	total: 3.62s	remaining: 4.97s
450:	learn: 0.3449546	test: 0.3823755	best: 0.3823524 (449)	total: 5.4s	remaining: 3.15s
600:	learn: 0.3285484	test: 0.3786046	best: 0.3786046 (600)	total: 7.15s	remaining: 1.34s


[I 2025-03-06 21:13:23,160] Trial 1 finished with value: 0.8275862068965517 and parameters: {'n_estimators': 714, 'learning_rate': 0.02411184060751367, 'depth': 5, 'l2_leaf_reg': 0.002656300020029928, 'random_strength': 0.004568952284776636, 'bagging_temperature': 0.27283651937346454, 'border_count': 41}. Best is trial 1 with value: 0.8275862068965517.


...

779:	learn: 0.3047520	test: 0.3806807	best: 0.3803102 (750)	total: 11.3s	remaining: 0us

bestTest = 0.3803101691
bestIteration = 750

Shrink model to first 751 iterations.
0:	learn: 0.6359836	test: 0.6402242	best: 0.6402242 (0)	total: 27.4ms	remaining: 10.3s
150:	learn: 0.2761339	test: 0.3659664	best: 0.3658150 (144)	total: 5.86s	remaining: 8.73s
300:	learn: 0.2135716	test: 0.3709761	best: 0.3569402 (214)	total: 9.78s	remaining: 2.44s


[I 2025-03-06 21:24:28,254] Trial 48 finished with value: 0.7931034482758621 and parameters: {'n_estimators': 376, 'learning_rate': 0.10264159841811965, 'depth': 8, 'l2_leaf_reg': 4.119746019318205, 'random_strength': 0.015255678890046845, 'bagging_temperature': 0.5042833546728485, 'border_count': 207}. Best is trial 31 with value: 0.8390804597701149.


375:	learn: 0.1909734	test: 0.3711251	best: 0.3569402 (214)	total: 11.8s	remaining: 0us

bestTest = 0.3569402315
bestIteration = 214

Shrink model to first 215 iterations.
0:	learn: 0.6790923	test: 0.6822811	best: 0.6822811 (0)	total: 24.1ms	remaining: 6.54s
150:	learn: 0.3384362	test: 0.3967095	best: 0.3967095 (150)	total: 3.37s	remaining: 2.7s


[I 2025-03-06 21:24:36,503] Trial 49 finished with value: 0.8160919540229885 and parameters: {'n_estimators': 272, 'learning_rate': 0.020524408619290405, 'depth': 8, 'l2_leaf_reg': 0.002242040503655568, 'random_strength': 0.08570674650771491, 'bagging_temperature': 0.2202144741280564, 'border_count': 120}. Best is trial 31 with value: 0.8390804597701149.


271:	learn: 0.2876124	test: 0.3785198	best: 0.3775135 (267)	total: 7.98s	remaining: 0us

bestTest = 0.3775134603
bestIteration = 267

Shrink model to first 268 iterations.


In [23]:
# Выводим лучшую попытку
trial = study.best_trial
print(f"Accuracy: {trial.value}")

print("Параметры: ")
for key, val in trial.params.items():
    print(f"{key}: {val}")

Accuracy: 0.8390804597701149
Параметры: 
n_estimators: 242
learning_rate: 0.03936393479278138
depth: 8
l2_leaf_reg: 0.0021756837351462124
random_strength: 0.06896444475131593
bagging_temperature: 0.2760190266969499
border_count: 161


In [24]:
best_params = trial.params
best_params['cat_features'] = cat_features
best_params['random_state'] = SEED
best_params['verbose'] = False

In [25]:
clf = CatBoostClassifier(**best_params)
clf.fit(
    X_train,
    y_train,
    eval_set=(X_test, y_test),
    verbose=150,
    plot=False
)

0:	learn: 0.6671748	test: 0.6735507	best: 0.6735507 (0)	total: 27.3ms	remaining: 6.57s
150:	learn: 0.2848586	test: 0.3724627	best: 0.3678246 (146)	total: 3.75s	remaining: 2.26s
241:	learn: 0.2361476	test: 0.3742140	best: 0.3651830 (162)	total: 6.03s	remaining: 0us

bestTest = 0.3651829661
bestIteration = 162

Shrink model to first 163 iterations.


# Сабмит

## Подготовка данных

In [26]:
test = process_features(test)

Пропуски

In [27]:
test = replace_nans(test)

<ipython-input-17-e085b354fe4b>:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[feature] = df[feature].fillna(train[feature].mode()[0])


In [28]:
test.isna().sum().sort_values(ascending=False)

,0
HomePlanet,0
PassengerNumberInGroup,0
IsTravelWithFamily,0
IsAlone,0
PassengersCountInGroup,0
PaidExtra,0
PaidSum,0
Side,0
Num,0
Deck,0


In [29]:
test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,PassengerNumberInGroup,Deck,Num,Side,PaidSum,PaidExtra,PassengersCountInGroup,IsAlone,IsTravelWithFamily,Gender
0,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,...,1,G,3.0,S,0.0,False,1,True,False,female
1,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,...,1,F,4.0,S,2832.0,True,1,True,False,unknown
2,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,...,1,C,0.0,S,0.0,False,1,True,False,unknown
3,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,...,1,C,1.0,S,7418.0,True,1,True,False,unknown
4,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,...,1,F,5.0,S,645.0,True,1,True,False,unknown


## Сам сабмит

In [30]:
clf.predict(test[filtered_features])

array([ True, False,  True, ...,  True,  True,  True])

In [31]:
def save_submit():
    current_time = datetime.datetime.now(pytz.timezone('Asia/Krasnoyarsk'))
    test[['PassengerId', 'Transported']].to_csv(os.path.join(SUBS_PATH, 'sub_' + current_time.strftime("%d_%B_%H:%M") + '.csv'), index=False)

In [32]:
def save_notebook():
    current_time = datetime.datetime.now(pytz.timezone('Asia/Krasnoyarsk'))
    new_name = 'nb_' + current_time.strftime("%d_%B_%H:%M") + '.ipynb'

    notebook_path = os.path.join(DRIVE_PATH, 'solution.ipynb')

    destination_path = f"{NOTEBOOKS_PATH}/{new_name}.ipynb"
    shutil.copy(notebook_path, destination_path)

In [33]:
test['PassengerId'] = pd.read_csv(os.path.join(DATA_PATH, TEST_FILE_NAME))['PassengerId']
test['Transported'] = pd.DataFrame(clf.predict(test[filtered_features]))

In [34]:
save_submit()
save_notebook()

# Auto downloading
test[['PassengerId', 'Transported']].to_csv('submit.csv', index=False)
files.download('submit.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Сохранение модели

In [36]:
import pickle

with open('model.pkl', 'wb') as model_pkl:
  pickle.dump(clf, model_pkl)